<a href="https://colab.research.google.com/github/paulinaeb/IDaSec-project/blob/exp3/my_exp/before_evasion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --branch main https://github.com/paulinaeb/IDaSec-project.git

fatal: destination path 'IDaSec-project' already exists and is not an empty directory.


Libraries

In [13]:
!pip uninstall -y torch torchtext


Found existing installation: torch 2.0.1
Uninstalling torch-2.0.1:
  Successfully uninstalled torch-2.0.1
Found existing installation: torchtext 0.15.2
Uninstalling torchtext-0.15.2:
  Successfully uninstalled torchtext-0.15.2


In [14]:
# !pip install --upgrade torchtext
# !pip install --upgrade torch
!pip install torch==2.2.0 torchtext==0.17.0




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 990.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import os
import pandas as pd
import torch

# nlp library of Pytorch
from torchtext import data

import warnings as wrn
wrn.filterwarnings('ignore')


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

In [3]:
# Path to the dataset folder
dataset_path = "IDaSec-project/dataset"

# List all subfolders in dataset
subfolders = [f.name for f in os.scandir(dataset_path) if f.is_dir()]
print("available datasets:", subfolders)


available datasets: ['enron1', 'sms', 'enron2']


In [4]:
enron1_path = dataset_path + "/enron1"
enron2_path = dataset_path + "/enron2"
sms_path = dataset_path + "/sms"

In [5]:
enron1_files = os.listdir(enron1_path)
print(f"\nFiles in '{enron1_path}': {enron1_files}")

enron2_files = os.listdir(enron2_path)
print(f"\nFiles in '{enron2_path}': {enron2_files}")

sms_files = os.listdir(sms_path)
print(f"\nFiles in '{sms_path}': {sms_files}")


Files in 'IDaSec-project/dataset/enron1': ['enron1_train.csv', 'enron1_val.csv', 'enron1_test.csv']

Files in 'IDaSec-project/dataset/enron2': ['enron2_val.csv', 'enron2_train.csv', 'enron2_test.csv']

Files in 'IDaSec-project/dataset/sms': ['val.csv', 'train.csv', 'test.csv']


Enron1

In [6]:
enron1_train = pd.read_csv(enron1_path + "/enron1_train.csv")
enron1_test = pd.read_csv(enron1_path + "/enron1_test.csv")
enron1_val = pd.read_csv(enron1_path + "/enron1_val.csv")

In [9]:
enron1_train.head()

,email,target
0,"Subject: prom dress shopping hi , just wanted ...",ham
1,"Subject: hi agaain hello , welcome to pharm la...",spam
2,Subject: feedback monitor error - meter 984132...,ham
3,Subject: instructions to remove spyware / adwa...,spam
4,Subject: acrobat pro 7 . 0 $ 69 . 95 xp pro op...,spam


The model

In [7]:
SEED = 42

torch.manual_seed(SEED)
torch.backends.cuda.deterministic = True

In [9]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence


In [23]:
tokenizer = get_tokenizer("spacy", language="en_core_web_sm")

# Example: assuming your dataset has a 'text' column and a 'type' label (spam/ham)
def yield_tokens(data_iter):
  for index, row in data_iter.iterrows():
        yield tokenizer(row['email'])


In [12]:
enron1_train.head()

,email,target
0,"Subject: prom dress shopping hi , just wanted ...",ham
1,"Subject: hi agaain hello , welcome to pharm la...",spam
2,Subject: feedback monitor error - meter 984132...,ham
3,Subject: instructions to remove spyware / adwa...,spam
4,Subject: acrobat pro 7 . 0 $ 69 . 95 xp pro op...,spam


In [24]:
vocab = build_vocab_from_iterator(yield_tokens(enron1_train), specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])

# Optionally map labels (e.g., ham → 0, spam → 1)
label_map = {"ham": 0, "spam": 1}


In [25]:
class TextDataset(Dataset):
    def __init__(self, data, vocab, tokenizer, label_map):
        self.data = data
        self.vocab = vocab
        self.tokenizer = tokenizer
        self.label_map = label_map

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = torch.tensor(self.vocab(self.tokenizer(item['text'])), dtype=torch.long)
        label = torch.tensor(self.label_map[item['type']], dtype=torch.float)
        return text, label


In [26]:
def collate_batch(batch):
    texts, labels = zip(*batch)
    lengths = torch.tensor([len(t) for t in texts])
    padded_texts = pad_sequence(texts, batch_first=True, padding_value=vocab["<pad>"])
    labels = torch.stack(labels)
    return padded_texts, lengths, labels


In [28]:
train_dataset = TextDataset(enron1_train, vocab, tokenizer, label_map)
val_dataset = TextDataset(enron1_val, vocab, tokenizer, label_map)
test_dataset = TextDataset(enron1_test, vocab, tokenizer, label_map)

BATCH_SIZE = 64

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)


In [ ]:
import torch.nn as nn

class LSTMNet(nn.Module):

    def __init__(self,vocab_size,embedding_dim,hidden_dim,output_dim,n_layers,bidirectional,dropout):

        super(LSTMNet,self).__init__()

        # Embedding layer converts integer sequences to vector sequences
        self.embedding = nn.Embedding(vocab_size,embedding_dim)

        # LSTM layer process the vector sequences
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            num_layers = n_layers,
                            bidirectional = bidirectional,
                            dropout = dropout,
                            batch_first = True
                           )

        # Dense layer to predict
        self.fc = nn.Linear(hidden_dim * 2,output_dim)
        # Prediction activation function
        self.sigmoid = nn.Sigmoid()


    def forward(self,text,text_lengths):
        embedded = self.embedding(text)
        # Thanks to packing, LSTM don't see padding tokens
        # and this makes our model better
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(),batch_first=True)

        packed_output,(hidden_state,cell_state) = self.lstm(packed_embedded)

        # Concatenating the final forward and backward hidden states
        hidden = torch.cat((hidden_state[-2,:,:], hidden_state[-1,:,:]), dim = 1)

        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.sigmoid(dense_outputs)

        return outputs



In [ ]:
SIZE_OF_VOCAB = len(TEXT.vocab)
EMBEDDING_DIM = 100
NUM_HIDDEN_NODES = 64
NUM_OUTPUT_NODES = 1
NUM_LAYERS = 2
BIDIRECTION = True
DROPOUT = 0.2

In [ ]:
model = LSTMNet(SIZE_OF_VOCAB,
                EMBEDDING_DIM,
                NUM_HIDDEN_NODES,
                NUM_OUTPUT_NODES,
                NUM_LAYERS,
                BIDIRECTION,
                DROPOUT
               )

In [ ]:
import torch.optim as optim
model = model.to(device)
optimizer = optim.Adam(model.parameters(),lr=1e-4)
criterion = nn.BCELoss()
criterion = criterion.to(device)